In [1]:
import boto3
import sagemaker
# Specify your AWS Region
aws_region='us-east-1'

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Role to give SageMaker permission to access AWS services.
sagemaker_role = sagemaker.get_execution_role()


## Create or use the image

In [2]:
from sagemaker import image_uris

# Specify an AWS container image. 
container = image_uris.retrieve(
                                region=boto3.Session().region_name,
                                framework="linear-learner"
                                )

## Get the model artifact

In [3]:
# Create a variable w/ the model S3 URI
# First, provide the name of your S3 bucket
s3_bucket = 'sagemaker-us-east-1-410677554255' 

# Specify what directory within your S3 bucket your model is stored in
bucket_prefix = 'linear-learner/output/linear-learner-2022-11-10-15-56-58-055/output/'


# Replace with the name of your model artifact
model_filename = 'model.tar.gz'

# Relative S3 path
model_s3_key = f'{bucket_prefix}/'+model_filename

# Combine bucket name, model file name, and relate S3 path to create S3 model URI
model_url = 's3://sagemaker-us-east-1-410677554255/linear-learner/output/linear-learner-2022-11-10-15-56-58-055/output/model.tar.gz'                            

# Bring the model

In [5]:
from sagemaker.predictor import RealTimePredictor, csv_serializer, csv_deserializer

class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session=None):
        super(Predictor, self).__init__(
            endpoint_name, sagemaker_session, csv_serializer, csv_deserializer
        )



In [6]:
model = sagemaker.model.Model(
    predictor_cls=Predictor,
    image_uri=container,
    model_data=model_url,
    role=sagemaker_role)

## Load the data

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

# get the data
RANDOM_SEED = 42
dataset = 'keypoint.csv'
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=(0))

X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Deploy the estimator to an endpoint

In [8]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = model.deploy(instance_type='ml.t2.medium',
                                     initial_instance_count=1)
# cHEAPER ml.m6g.medium 0.0462
# ml.t2.medium 0.056

--------------------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Inference

In [14]:
endpoint_name = 'linear-learner-2022-11-15-13-56-26-646'
# Your labels
label_names = ['Open','Close','Pointer','OK']
# Data for inference
data = X_test[120]
result = predictor.predict(data=data)
print(result)
#print(label_names[np.argmax(result['predictions'])])

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[['2']]


In [15]:
y_test[120]


2.0